<a href="https://colab.research.google.com/github/ArghyaPal/Adversarial-Data-Programming/blob/master/ADP_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adversarial Data Programming: Using GANs to Relax the Bottleneck of Curated Labeled Data

Paucity of large curated hand-labeled training data forms a major bottleneck in the deployment of machine learning models in computer vision and other fields. Recent work (Data Programming) has shown how distant su- pervision signals in the form of labeling functions can be used to obtain labels for given data in near-constant time.

In this work, we present Adversarial Data Programming (ADP), which presents an adversarial methodology to gen- erate data as well as a curated aggregated label, given a set of weak labeling functions. We validated our method on the MNIST, Fashion MNIST, CIFAR 10 and SVHN datasets, and it outperformed many state-of-the-art models. We con- ducted extensive experiments to study its usefulness, as well as showed how the proposed ADP framework can be used for transfer learning as well as multi-task learning, where data from two domains are generated simultaneously using the framework along with the label information. Our fu- ture work will involve understanding the theoretical impli- cations of this new framework from a game-theoretic per- spective, as well as explore the performance of the method on more complex datasets.

## Import all necessary modules

In [0]:
import argparse
import os
import numpy as np
import math
import scipy
import itertools

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [0]:
os.makedirs("images", exist_ok=True)


n_epochs=200 #number of epochs of training
batch_size=32 #"size of the batches")
lr=0.0002 #"adam: learning rate")
b1=0.5 #"adam: decay of first order momentum of gradient")
b2=0.999  #"adam: decay of first order momentum of gradient")
n_cpu=8  #number of cpu threads to use during batch generation
latent_dim=100   #Dimensionality of the latent space
img_size=28  #size of each image dimension
channels=1 #number of image channels
sample_interval=400 #interval betwen image samples
n_classes=10       #number of classes

img_shape = (channels, img_size, img_size)

cuda = True if torch.cuda.is_available() else False


def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Linear") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [4]:
# Configure data loader
os.makedirs("./data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "./data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=batch_size,
    shuffle=True,
)

  0%|          | 16384/9912422 [00:00<01:30, 109192.23it/s]

9920512it [00:00, 29042005.64it/s]                           


Extracting ./data/mnist/MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 443676.35it/s]
  1%|          | 16384/1648877 [00:00<00:11, 142938.66it/s]

Extracting ./data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 7502637.18it/s]                            
8192it [00:00, 171639.92it/s]


Extracting ./data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.init_size = img_size // 4
        self.fc = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size ** 2))

        self.shared_conv = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
        )
        self.G1 = nn.Sequential(
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )
        self.G2_1 = nn.Sequential(
            nn.Conv2d(128, 100, 11, stride=1, padding=1),
            nn.BatchNorm2d(100, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(100, 60, 7, stride=1, padding=1),
            nn.BatchNorm2d(60, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(60, 30, 7, stride=1, padding=1),
            nn.Tanh(),
            nn.Conv2d(30, 16, 7, stride=1, padding=1),
            nn.Tanh(),
            nn.Conv2d(16, 8, 7, stride=1, padding=1),
            nn.Tanh(),
            nn.Conv2d(8, 8, 5, stride=1, padding=1),
            nn.Tanh(),
            nn.Conv2d(8, channels, 4, stride=1, padding=1),
            nn.Tanh(),
            nn.Conv2d(channels, channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, noise):
        out = self.fc(noise)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img_emb = self.shared_conv(out)
        img = self.G1(img_emb)
        label = self.G2_1(img_emb)
        print("Label", label.shape)
        label = label.view(-1*1*1*1)
        return img, label

In [0]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.label_embedding = nn.Embedding(n_classes, n_classes)

        self.model = nn.Sequential(
            nn.Linear(n_classes + int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1),
        )

    def forward(self, img, labels):
        # Concatenate label embedding and image to produce input
        d_in = torch.cat((img.view(img.size(0), -1), self.label_embedding(labels)), -1)
        validity = self.model(d_in)
        return validity


In [0]:
# Loss functions
adversarial_loss = torch.nn.MSELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

In [0]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

In [0]:
def sample_image(n_row, batches_done):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    z = Variable(FloatTensor(np.random.normal(0, 1, (n_row ** 2, latent_dim))))
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([num for _ in range(n_row) for num in range(n_row)])
    labels = Variable(LongTensor(labels))
    gen_imgs = generator(z)
    save_image(gen_imgs, "images/%d.png" % batches_done, nrow=n_row, normalize=True)


In [0]:
# ----------
#  Training
# ----------

for epoch in range(n_epochs):
    for i, (imgs, labels) in enumerate(dataloader):

        batch_size = imgs.shape[0]

        # Adversarial ground truths
        valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
        fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(FloatTensor))
        labels = Variable(labels.type(LongTensor))
        
        print("Real", real_imgs.shape, labels.shape)

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise and labels as generator input
        z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, latent_dim))))

        # Generate a batch of images
        gen_imgs, gen_labels = generator(z)
        
        # Configure input
        gen_imgs = Variable(gen_imgs.type(FloatTensor))
        gen_labels = Variable(gen_labels.type(LongTensor))
        
        print("Generated", gen_imgs.shape, gen_labels.shape)

        # Loss measures generator's ability to fool the discriminator
        validity = discriminator(gen_imgs, gen_labels)
        g_loss = adversarial_loss(validity, valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Loss for real images
        validity_real = discriminator(real_imgs, labels)
        d_real_loss = adversarial_loss(validity_real, valid)

        # Loss for fake images
        validity_fake = discriminator(gen_imgs.detach(), gen_labels)
        d_fake_loss = adversarial_loss(validity_fake, fake)

        # Total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        #batches_done = epoch * len(dataloader) + i
        #if batches_done % sample_interval == 0:
        #    sample_image(n_row=10, batches_done=batches_done)

Real torch.Size([32, 1, 28, 28]) torch.Size([32])
Label torch.Size([32, 1, 1, 1])
Generated torch.Size([32, 1, 28, 28]) torch.Size([32])
[Epoch 0/200] [Batch 0/1875] [D loss: 0.220355] [G loss: 1.031277]
Real torch.Size([32, 1, 28, 28]) torch.Size([32])
Label torch.Size([32, 1, 1, 1])
Generated torch.Size([32, 1, 28, 28]) torch.Size([32])
[Epoch 0/200] [Batch 1/1875] [D loss: 0.069267] [G loss: 1.010162]
Real torch.Size([32, 1, 28, 28]) torch.Size([32])
Label torch.Size([32, 1, 1, 1])
Generated torch.Size([32, 1, 28, 28]) torch.Size([32])
[Epoch 0/200] [Batch 2/1875] [D loss: 0.018744] [G loss: 0.991013]
Real torch.Size([32, 1, 28, 28]) torch.Size([32])
Label torch.Size([32, 1, 1, 1])
Generated torch.Size([32, 1, 28, 28]) torch.Size([32])
[Epoch 0/200] [Batch 3/1875] [D loss: 0.018358] [G loss: 0.970787]
Real torch.Size([32, 1, 28, 28]) torch.Size([32])
Label torch.Size([32, 1, 1, 1])
Generated torch.Size([32, 1, 28, 28]) torch.Size([32])
[Epoch 0/200] [Batch 4/1875] [D loss: 0.016674]